# Introduction
This project is purely for fun and for people who want to find good restaurants in their city like I am. 
So i will restrain city to chicago. 

Also, this project will explore both recommenderlab and Lightfm to compare their performances.

In [5]:
## packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score
import scipy
import simplejson as json


In [6]:
## load data
cities = []
path = '/Users/ivanchen/Desktop/MSIA/Studying_materials/yelp_dataset/business.json'
with open(path) as fin:
    i = 0 
    for line in fin:
        line_contents = json.loads(line)
        categories = line_contents['categories']
        city = line_contents['city']
        try:
            if 'Restaurants' in categories or 'Food' in categories:
                cities.append(city)
            if 'Chicago' == city:
                print(line_contents)
                
        except TypeError:
            #print(line_contents)
            #print(city)
            pass



In [7]:
a = pd.Series(cities)

In [8]:
b = pd.DataFrame({'city' : a})
c = b.groupby('city').size().reset_index()
c.columns =  ['city','count']

In [10]:
c.sort_values(by = 'count',ascending = False).head()


,city,count
758,Toronto,10093
326,Las Vegas,8286
552,Phoenix,5130
439,Montréal,4540
93,Calgary,3694


In [11]:
# fix for toronto right now
mydata = []
with open(path) as fin:
    i = 0 
    for line in fin:
        line_contents = json.loads(line)
        categories = line_contents['categories']
        city = line_contents['city']
        try:
            if ('Restaurants' in categories or 'Food' in categories) and 'Toronto' == city:
                mydata.append(line_contents)
                
        except TypeError:
            #print(line_contents)
            print(city)


Scottsdale
Airdrie
Pittsburgh
Charlotte
Etobicoke
Tempe
Ajax
North Las Vegas
Scottsdale
Cleveland
Madison
Phoenix
Toronto
North Las Vegas
Kannapolis
Las Vegas
Brampton
North Las Vegas
Glendale
Phoenix
Mississauga
Las Vegas
Charlotte
Toronto
Mississauga
Avondale
Phoenix
Henderson
Phoenix
Las Vegas
Phoenix
Cleveland
Gilbert
Montréal
Las Vegas
Toronto
Cleveland
Cleveland
Glendale
Henderson
Scottsdale
Phoenix
Mesa
Phoenix
Phoenix
Phoenix
Phoenix
Las Vegas
Chandler
Mississauga
Montréal
Laveen
Toronto
Phoenix
Matthews
North York
Charlotte
Las Vegas
Phoenix
Bolton
Phoenix
Charlotte
Henderson
Phoenix
Rocky River
Montréal
AZ
Las Vegas
Cheswick
Glendale
Beloeil
Phoenix
Mesa
Avondale
Chandler
Charlotte
Beloeil
Cleveland
Scottsdale
Las Vegas
Akron
North Las Vegas
Pittsburgh
Montréal
Las Vegas
Montréal
Las Vegas
Strongsville
Verona
Charlotte
Scottsdale
Las Vegas
Willoughby
Tempe
Champaign
Tempe
Cave Creek
Phoenix
Charlotte
Montréal
Las Vegas
Charlotte
Mesa
Turtle Creek
Las Vegas
Chandler
North Las 

In [65]:
mydata[1:50]

[{'business_id': 'NDuUMJfrWk52RA-H-OtrpA',
  'name': 'Bolt Fresh Bar',
  'address': '1170 Queen Street W',
  'city': 'Toronto',
  'state': 'ON',
  'postal_code': 'M6J 1J5',
  'latitude': 43.6428886,
  'longitude': -79.4254291,
  'stars': 3.0,
  'review_count': 57,
  'is_open': 1,
  'attributes': {'WiFi': "u'no'",
   'BikeParking': 'True',
   'RestaurantsPriceRange2': '2',
   'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
   'RestaurantsTakeOut': 'True',
   'Caters': 'False'},
  'categories': 'Juice Bars & Smoothies, Food, Restaurants, Fast Food, Vegan',
  'hours': {'Monday': '8:0-21:0',
   'Tuesday': '8:0-21:0',
   'Wednesday': '8:0-21:0',
   'Thursday': '8:0-21:0',
   'Friday': '8:0-21:0',
   'Saturday': '9:0-21:0',
   'Sunday': '9:0-21:0'}},
 {'business_id': 'SP_YXIEwkFPPl_9anCYmpQ',
  'name': 'The Steady Cafe & Bar',
  'address': '1051 Bloor Street W',
  'city': 'Toronto',
  'state': 'ON',
  'postal_code': 'M6H 1M4',
  'lati

In [12]:
column_names = mydata[1].keys()
for item in mydata:
    
    if len([x for x in item.keys() if x in column_names]) != len(item.keys()):
        print(item.keys())


In [13]:
data = []
for column in column_names:
    this_col = []
    for item in mydata:
        this_col.append(item[column])
    data.append(this_col)


In [18]:
np.array(data[12])

array(['Bakeries, Food',
       'Juice Bars & Smoothies, Food, Restaurants, Fast Food, Vegan',
       'Restaurants, Nightlife, Breakfast & Brunch, Vegan, Food, Tapas Bars, Bars, Southern, Coffee & Tea',
       ..., 'Coffee & Tea, Food',
       'Korean, Japanese, Asian Fusion, Restaurants', 'Restaurants, Thai'],
      dtype='<U257')

In [19]:
column_names

dict_keys(['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours'])

In [36]:
first_cols = list(column_names)[0:10]
#first_cols.append(list(column_names)[12])
pd_data ={}
idx = 0
for column in first_cols:
    pd_data[column] = data[idx]
    idx+=1
business = pd.DataFrame(pd_data)
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count
0,C9oCPomVP0mtKa8z99E3gg,Bakery Gateau,"865 York Mills Road, Unit 1",Toronto,ON,M3B 1Y6,43.754093,-79.349548,4.5,8
1,NDuUMJfrWk52RA-H-OtrpA,Bolt Fresh Bar,1170 Queen Street W,Toronto,ON,M6J 1J5,43.642889,-79.425429,3.0,57
2,SP_YXIEwkFPPl_9anCYmpQ,The Steady Cafe & Bar,1051 Bloor Street W,Toronto,ON,M6H 1M4,43.660494,-79.432099,3.5,29
3,mlHC2XcU9Bows6cnYEmRgg,Mad Crush Wine Bar,582 College Street,Toronto,ON,M6G 1B3,43.655420,-79.413352,4.0,9
4,RFbMVekR8lU9tPJ8sWrwHg,Tavolino,619 Mount Pleasant Road,Toronto,ON,M4S 2M5,43.704229,-79.388230,4.0,18


In [37]:
business['category'] = data[12]
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,category
0,C9oCPomVP0mtKa8z99E3gg,Bakery Gateau,"865 York Mills Road, Unit 1",Toronto,ON,M3B 1Y6,43.754093,-79.349548,4.5,8,"Bakeries, Food"
1,NDuUMJfrWk52RA-H-OtrpA,Bolt Fresh Bar,1170 Queen Street W,Toronto,ON,M6J 1J5,43.642889,-79.425429,3.0,57,"Juice Bars & Smoothies, Food, Restaurants, Fas..."
2,SP_YXIEwkFPPl_9anCYmpQ,The Steady Cafe & Bar,1051 Bloor Street W,Toronto,ON,M6H 1M4,43.660494,-79.432099,3.5,29,"Restaurants, Nightlife, Breakfast & Brunch, Ve..."
3,mlHC2XcU9Bows6cnYEmRgg,Mad Crush Wine Bar,582 College Street,Toronto,ON,M6G 1B3,43.655420,-79.413352,4.0,9,"Restaurants, Breakfast & Brunch, Bars, Modern ..."
4,RFbMVekR8lU9tPJ8sWrwHg,Tavolino,619 Mount Pleasant Road,Toronto,ON,M4S 2M5,43.704229,-79.388230,4.0,18,"Italian, Restaurants"


In [50]:
#get all nested columns
def get_superset_of_column_names_from_file(data,parent_key):
    """Read in the json dataset file and return the superset of column names."""
    column_names = set()
    
    for line_contents in data:
        
        column_names.update(
                set(get_column_names(line_contents,parent_key).keys())
                )
    return column_names

def get_column_names(line_contents, parent_key):
    """Return a list of flattened key names given a dict.
    Example:
        line_contents = {
            'a': {
                'b': 2,
                'c': 3,
                },
        }
        will return: ['a.b', 'a.c']
    These will be the column names for the eventual csv file.
    """
    column_names = []
    for k in line_contents.keys():
        column_names.append('{0}.{1}'.format(parent_key,k)
        )
    return dict(column_names)


In [51]:
column_names = get_superset_of_column_names_from_file(data[11],'attribute')

ValueError: dictionary update sequence element #0 has length 29; 2 is required

In [44]:
def extract_nested_values(target_ls):
    col_dict = {}
    print('job length:' + str(len(target_ls)))
    for target in target_ls:
        for key in target.keys():
            if key in col_dict.keys():
                col_dict[key].append()
        

SyntaxError: unexpected EOF while parsing (<ipython-input-44-e1a36e8162f2>, line 6)

In [42]:
data[11][1]

{'WiFi': "u'no'",
 'BikeParking': 'True',
 'RestaurantsPriceRange2': '2',
 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
 'RestaurantsTakeOut': 'True',
 'Caters': 'False'}

In [54]:
data[11]

[{'RestaurantsDelivery': 'False',
  'RestaurantsPriceRange2': '2',
  'RestaurantsTakeOut': 'True',
  'BikeParking': 'True',
  'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"},
 {'WiFi': "u'no'",
  'BikeParking': 'True',
  'RestaurantsPriceRange2': '2',
  'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
  'RestaurantsTakeOut': 'True',
  'Caters': 'False'},
 {'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
  'RestaurantsReservations': 'True',
  'GoodForKids': 'False',
  'RestaurantsAttire': "u'casual'",
  'Alcohol': "u'full_bar'",
  'OutdoorSeating': 'False',
  'RestaurantsPriceRange2': '2',
  'NoiseLevel': "u'average'",
  'RestaurantsDelivery': 'False',
  'WiFi': "u'free'",
  'RestaurantsTakeOut': 'False',
  'HasTV': 'False',
  'BikeParking': 'True',
  'Ambience': "{'romantic': False, 'intimate': False, 'classy': Fal

In [43]:
data[11][2]

{'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
 'RestaurantsReservations': 'True',
 'GoodForKids': 'False',
 'RestaurantsAttire': "u'casual'",
 'Alcohol': "u'full_bar'",
 'OutdoorSeating': 'False',
 'RestaurantsPriceRange2': '2',
 'NoiseLevel': "u'average'",
 'RestaurantsDelivery': 'False',
 'WiFi': "u'free'",
 'RestaurantsTakeOut': 'False',
 'HasTV': 'False',
 'BikeParking': 'True',
 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}",
 'RestaurantsGoodForGroups': 'True',
 'Caters': 'True'}

In [46]:
data[11][3]

{'Alcohol': "u'full_bar'",
 'Caters': 'False',
 'Smoking': "u'no'",
 'WheelchairAccessible': 'True',
 'DogsAllowed': 'False',
 'CoatCheck': 'False',
 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
 'RestaurantsTakeOut': 'True',
 'WiFi': "u'free'",
 'RestaurantsTableService': 'True',
 'HasTV': 'False',
 'RestaurantsPriceRange2': '3',
 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}",
 'HappyHour': 'False',
 'RestaurantsReservations': 'True',
 'BikeParking': 'True',
 'RestaurantsAttire': "'dressy'",
 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}",
 'OutdoorSeating': 'True',
 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': False}",
 'RestaurantsDel

dict_keys(['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours'])

In [41]:
data[11][1]

{'WiFi': "u'no'",
 'BikeParking': 'True',
 'RestaurantsPriceRange2': '2',
 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
 'RestaurantsTakeOut': 'True',
 'Caters': 'False'}

In [52]:
a = pd.read_csv('/Users/ivanchen/Desktop/MSIA/Studying_materials/yelp_dataset/business.csv')
a.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,23,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,attributes.WiFi,attributes.BestNights,attributes.RestaurantsCounterService,attributes.RestaurantsPriceRange2,attributes.Alcohol,attributes.DogsAllowed,attributes.GoodForKids,attributes.WheelchairAccessible,attributes.BusinessAcceptsBitcoin,stars,...,hours.Wednesday,attributes.DietaryRestrictions,postal_code,attributes.Open24Hours,attributes.BusinessAcceptsCreditCards,attributes.Music,attributes.RestaurantsTakeOut,address,attributes.RestaurantsGoodForGroups,attributes.BikeParking
0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,3.0,...,NaN,NaN,85016,NaN,NaN,NaN,NaN,2818 E Camino Acequia Drive,NaN,NaN
1,u'no',NaN,NaN,2,u'full_bar',NaN,True,NaN,NaN,2.5,...,9:0-0:0,NaN,L5R 3E7,NaN,NaN,NaN,True,30 Eglinton Avenue W,True,False
2,u'no',NaN,NaN,2,u'beer_and_wine',NaN,True,NaN,NaN,4.0,...,17:30-21:30,NaN,28210,NaN,True,NaN,True,"10110 Johnston Rd, Ste 15",True,True
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,8:0-17:0,NaN,85338,NaN,NaN,NaN,NaN,"15655 W Roosevelt St, Ste 237",NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,4.0,...,7:0-23:0,NaN,28217,NaN,True,NaN,NaN,"4209 Stuart Andrew Blvd, Ste F",NaN,NaN
